- Jsonlを対象にした重複除去を行う
- https://github.com/if001/dedup_sentence.git を利用したC++実装のdeduplication(重複を除く)
- 巨大コーパスをまとめて処理するのは現実的でないので，分割したコーパスを扱えるプログラムを利用 (branch: multi_file)
- Colabより，Localマシンで行ったほうが処理時間は1/3だったので，Localマシンでの実行を要検討

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')
!ls

Mounted at /content/drive
drive  sample_data


In [ ]:
!apt install nlohmann-json3-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  nlohmann-json3-dev
0 upgraded, 1 newly installed, 0 to remove and 39 not upgraded.
Need to get 167 kB of archives.
After this operation, 1,247 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 nlohmann-json3-dev all 3.10.5-2 [167 kB]
Fetched 167 kB in 1s (211 kB/s)
Selecting previously unselected package nlohmann-json3-dev.
(Reading database ... 121753 files and directories currently installed.)
Preparing to unpack .../nlohmann-json3-dev_3.10.5-2_all.deb ...
Unpacking nlohmann-json3-dev (3.10.5-2) ...
Setting up nlohmann-json3-dev (3.10.5-2) ...


In [ ]:
# content以下にプログラムを配置. 一時的な置き場に保存しているため，ランタイムが切れたら削除される
%cd /content
!ls

/content
drive  sample_data


In [ ]:
!git clone https://github.com/if001/dedup_sentence.git
!cd dedup_sentence && git checkout multi_file   # 複数jsonlについてdedup可能なbranch
!pwd
!wget https://raw.githubusercontent.com/simdjson/simdjson/master/singleheader/simdjson.h
!wget https://raw.githubusercontent.com/simdjson/simdjson/master/singleheader/simdjson.cpp
!mv simdjson.h simdjson.cpp /content/dedup_sentence/.

Cloning into 'dedup_sentence'...
remote: Enumerating objects: 118, done.
remote: Counting objects: 100% (118/118), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 118 (delta 71), reused 83 (delta 36), pack-reused 0
Receiving objects: 100% (118/118), 22.20 KiB | 11.10 MiB/s, done.
Resolving deltas: 100% (71/71), done.
Branch 'multi_file' set up to track remote branch 'multi_file' from 'origin'.
Switched to a new branch 'multi_file'
/content
--2024-03-28 08:28:37--  https://raw.githubusercontent.com/simdjson/simdjson/master/singleheader/simdjson.h
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3840839 (3.7M) [text/plain]
Saving to: ‘simdjson.h’

simdjson.h          100%[===================>]   3.66M  --.-KB/s    in 0.07s   

2024-03-28 08

In [ ]:
# dedup_sentence 以下にsmhasherリポジトリを作成
# おそらくmake時にNUM_WORKERSを指定. default = 2, (おそらく，指定するならexport NUM_WORKERS=4 要テスト. コンパイル後に環境変数に指定してもプログラムへの反映はされない．テスト済)
%cd /content/dedup_sentence
!git clone https://github.com/aappleby/smhasher.git
!pwd
!ls
!make
!ls # -> deduplicate が作成

/content/dedup_sentence
Cloning into 'smhasher'...
remote: Enumerating objects: 516, done.
remote: Total 516 (delta 0), reused 0 (delta 0), pack-reused 516
Receiving objects: 100% (516/516), 269.61 KiB | 3.06 MiB/s, done.
Resolving deltas: 100% (363/363), done.
/content/dedup_sentence
dedup.cpp   Hasher.hpp	    Makefile	  simdjson.h  text.hpp
dedup.hpp   HasherTest.cpp  readme.md	  smhasher    ThreadPool.cpp
Hasher.cpp  main.cpp	    simdjson.cpp  text.cpp    ThreadPool.hpp
g++ -std=c++17 -Wall -Wextra -c main.cpp
main.cpp: In function ‘std::unordered_set<std::__cxx11::basic_string<char> > processFile(const string&, const string&, const string&, ThreadPool&, int)’:
main.cpp:35:14: warning: unused variable ‘error’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wunused-variable-Wunused-variable]8;;]
   35 |         auto error = doc["text"].get(res);
      |              ^~~~~
g++ -std=c++17 -Wall -Wextra -c Hasher.cpp
g++ -std=c++17 -Wall -Wextra -c text.cpp
g++ -c

In [ ]:
# /content/drive/MyDrive/Colab Notebooks/MyProjects/test_LLM  # このパスの下に処理したいdataなどを配置． (自環境に合わせて要変更)
# 以下に3つのDirを作成
# jsonl入力用Dir /content/drive/MyDrive/Colab Notebooks/MyProjects/test_LLM/test_jsonl     # 以下に1.jsonl, 2.jsonl, ... を配置
# jsonl出力用Dir /content/drive/MyDrive/Colab Notebooks/MyProjects/test_LLM/dedup_out
# プログラムで使用されるhash用Dir /content/drive/MyDrive/Colab Notebooks/MyProjects/test_LLM/dedup_hash


# Program Usage: ./deduplicate <start> <end> <inputDir> <outputDir> <processedHashesDir>

# inputDir確認 spaceは\でエスケープすること
!ls /content/drive/MyDrive/Colab\ Notebooks/MyProjects/test_LLM/test_jsonl/

# dedup 実行
# Colabでは, 137384line (627Mb) のjsonlを処理するのに90min かかった
!time ./deduplicate 1 1 /content/drive/MyDrive/Colab\ Notebooks/MyProjects/test_LLM/test_jsonl/ /content/drive/MyDrive/Colab\ Notebooks/MyProjects/test_LLM/dedup_out/ /content/drive/MyDrive/Colab\ Notebooks/MyProjects/test_LLM/dedup_hash

1.jsonl  results.filtering.jsonl
worker...2

Processing file: /content/drive/MyDrive/Colab Notebooks/MyProjects/test_LLM/test_jsonl//1.jsonl
start cal hash...
wait cal hash...
start check dedup other file...

Duplicated: 4060/137384
save blacklist file.../content/drive/MyDrive/Colab Notebooks/MyProjects/test_LLM/dedup_hash/1.txt

real	89m12.001s
user	143m44.074s
sys	0m29.288s


In [ ]:
!wc -l /content/drive/MyDrive/Colab\ Notebooks/MyProjects/test_LLM/dedup_out/1.jsonl

133324 /content/drive/MyDrive/Colab Notebooks/MyProjects/test_LLM/dedup_out/1.jsonl
